In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import skew, kurtosis
import warnings
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings('ignore')

In [3]:
# Configuration des graphiques
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

sns.set_palette("husl")

In [4]:
# Charger le fichier original
df = pd.read_csv('data/cleaned_paddydataset.csv', sep=',', encoding='utf-8', low_memory=False)
# Création d'une copie pour ne pas modifier df original
df_fe = df.copy()

In [5]:
# ----------------------------------------------------------------
# 3. DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES
# ----------------------------------------------------------------

print("\n" + "-"*70)
print("DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES")
print("-"*70)

# Identification des types de variables
numeric_features = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = df.select_dtypes(include=['object']).columns.tolist()

print(f"Variables numériques: {len(numeric_features)}")
print(f"  Exemples: {numeric_features[:10]}")
print(f"\nVariables catégoriques: {len(categorical_features)}")
print(f"  Exemples: {categorical_features[:10]}")


----------------------------------------------------------------------
DISTINCTION VARIABLES NUMÉRIQUES ET CATÉGORIQUES
----------------------------------------------------------------------
Variables numériques: 38
  Exemples: ['Hectares ', 'Seedrate(in Kg)', 'LP_Mainfield(in Tonnes)', 'Nursery area (Cents)', 'LP_nurseryarea(in Tonnes)', 'DAP_20days', 'Weed28D_thiobencarb', 'Urea_40Days', 'Potassh_50Days', 'Micronutrients_70Days']

Variables catégoriques: 8
  Exemples: ['Agriblock', 'Variety', 'Soil Types', 'Nursery', 'Wind Direction_D1_D30', 'Wind Direction_D31_D60', 'Wind Direction_D61_D90', 'Wind Direction_D91_D120']


In [6]:
# Création de features météo cumulées et moyennes
# Cumul de pluie sur tout le cycle
df_fe['Rain_1_70days'] = df_fe['30DRain( in mm)'] + df_fe['30_50DRain( in mm)'] + df_fe['51_70DRain(in mm)']

# Température moyenne globale
df_fe['Temp_mean_D1_D30'] = (df_fe['Min temp_D1_D30'] + df_fe['Max temp_D1_D30']) / 2


In [7]:
# Rendement par hectare
df_fe['Yield_per_Hectare'] = df_fe['Paddy yield(in Kg)'] / df_fe['Hectares ']

# Production probable vs superficie de pépinière
df_fe['LP_nursery_ratio'] = df_fe['LP_nurseryarea(in Tonnes)'] / df_fe['Nursery area (Cents)']

In [8]:
# Label Encoding
# Dictionnaire pour garder les encodeurs
le_dict = {}

for col in categorical_features:
    le = LabelEncoder()
    df_fe[col + '_enc'] = le.fit_transform(df_fe[col].astype(str))
    le_dict[col] = le  # Pour inverser si besoin

In [9]:
# Aperçu des premières lignes
print("APERÇU DES DONNÉES:")
print("-" * 30)
print(df_fe.head())
print()

APERÇU DES DONNÉES:
------------------------------
   Hectares      Agriblock      Variety Soil Types  Seedrate(in Kg)  \
0        6.0     cuddalore        co_43   alluvial            150.0   
1        6.0   kurinjipadi      ponmani       clay            150.0   
2        6.0  sankarapuram  delux ponni   alluvial            150.0   
3        6.0  kallakurichi        co_43       clay            150.0   
4        4.0  sankarapuram      ponmani   alluvial            150.0   

   LP_Mainfield(in Tonnes) Nursery  Nursery area (Cents)  \
0                     75.0     dry                 120.0   
1                     75.0     wet                 120.0   
2                     75.0     dry                 120.0   
3                     75.0     wet                 120.0   
4                     75.0     dry                 120.0   

   LP_nurseryarea(in Tonnes)  DAP_20days  ...  Yield_per_Hectare  \
0                        6.0       240.0  ...             5838.0   
1                        

In [10]:
# Vérifier les variables à encoder
print(f"Variables catégoriques à encoder : {categorical_features}")

# ==============================
# One-Hot Encoding
# ==============================
df_fe = pd.get_dummies(df_fe, columns=categorical_features, drop_first=False)

print(f"Dimension du dataset après encodage : {df_fe.shape}")

# vérifier les premières colonnes
df_fe.head()

Variables catégoriques à encoder : ['Agriblock', 'Variety', 'Soil Types', 'Nursery', 'Wind Direction_D1_D30', 'Wind Direction_D31_D60', 'Wind Direction_D61_D90', 'Wind Direction_D91_D120']
Dimension du dataset après encodage : (2789, 85)


,Hectares,Seedrate(in Kg),LP_Mainfield(in Tonnes),Nursery area (Cents),LP_nurseryarea(in Tonnes),DAP_20days,Weed28D_thiobencarb,Urea_40Days,Potassh_50Days,Micronutrients_70Days,...,Wind Direction_D61_D90_nne,Wind Direction_D61_D90_nnw,Wind Direction_D61_D90_se,Wind Direction_D61_D90_sw,Wind Direction_D91_D120_nnw,Wind Direction_D91_D120_nw,Wind Direction_D91_D120_s,Wind Direction_D91_D120_sse,Wind Direction_D91_D120_w,Wind Direction_D91_D120_wsw
0,6.0,150.0,75.0,120.0,6.0,240.0,12.0,162.78,62.28,60.0,...,False,True,False,False,False,False,False,False,False,True
1,6.0,150.0,75.0,120.0,6.0,240.0,12.0,162.78,62.28,90.0,...,False,False,True,False,False,False,False,True,False,False
2,6.0,150.0,75.0,120.0,6.0,240.0,12.0,162.78,62.28,90.0,...,True,False,False,False,False,False,False,False,True,False
3,6.0,150.0,75.0,120.0,6.0,240.0,12.0,162.78,62.28,90.0,...,False,False,True,False,False,True,False,False,False,False
4,4.0,150.0,75.0,120.0,6.0,240.0,12.0,162.78,62.28,90.0,...,False,False,False,True,False,True,False,False,False,False


In [11]:
df_fe.to_csv('data/paddy_dataset_fe.csv', index=False)
print("Dataset avec features engineering enregistré dans 'data/paddy_dataset_fe.csv'")

Dataset avec features engineering enregistré dans 'data/paddy_dataset_fe.csv'
